# 2. CML API를 사용하여 Chroma DB에 데이터를 입력하는 Job 정의
이전 연습에서 선후행 관계로 정의된 job을 추가하는 연습을 UI에서 진행하였습니다. 이 노트북 에서는 CML APIv2를 사용하여 동일한 작업을 수행합니다. CML API를 사용하여 작업을 만드는 이점은 사용자가 프로그래밍 방식으로 작업을 만든 다음 실행할 수 있다는 것입니다. cmlapi 라이브러리를 사용하여 job 만드는 것은 자동화, 버전 제어, 재현성, 통합, 확장성, 오류 처리 및 작업 관리 효율성을 가능하게 하여 데이터 처리 워크플로를 간소화하기 때문에 유익합니다.

![Populate Chroma 아키텍처](../assets/exercise_5.png)

#### 2.1 CML API 클라이언트 생성 및 사용 가능한 런타임 나열
필요한 모듈을 가져오고, 컬렉션 이름을 정의하고, CML 클라이언트를 초기화하고, 특정 기준과 일치하는 사용 가능한 런타임 목록을 검색하여 사용 가능한 런타임 목록을 인쇄합니다.

In [2]:
import os
import cmlapi
import random
import string
import json

COLLECTION_NAME = 'cml-default' ## Update if you have changed this
    
client = cmlapi.default_client(url=os.getenv("CDSW_API_URL").replace("/api/v1", ""), cml_api_key=os.getenv("CDSW_APIV2_KEY"))
available_runtimes = client.list_runtimes(search_filter=json.dumps({
    "kernel": "Python 3.10",
    "edition": "Nvidia GPU",
    "editor": "JupyterLab",
    "full_version": "2024.05.1-b8"
}))
selected_runtime = max(available_runtimes.runtimes)
selected_runtime

{'description': 'Python runtime with CUDA libraries provided by Cloudera',
 'edition': 'Nvidia GPU',
 'editor': 'JupyterLab',
 'full_version': '2024.05.1-b8',
 'image_identifier': 'docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.10-cuda:2024.05.1-b8',
 'kernel': 'Python 3.10',
 'register_user_id': 0,
 'status': 'ENABLED'}

#### 2.2 지정된 ML 런타임 식별자를 검색하여 환경 변수에 저장합니다.

In [3]:
## Set available runtimes to the latest runtime in the environment (iterator is the number that begins with 0 and advances sequentially)
## The JOB_IMAGE_ML_RUNTIME variable stores the ML Runtime which will be used to launch the job
print(selected_runtime.image_identifier)
JOB_IMAGE_ML_RUNTIME = selected_runtime.image_identifier

## Store the ML Runtime for any future jobs in an environment variable so we don't have to do this step again
os.environ['JOB_IMAGE_ML_RUNTIME'] = JOB_IMAGE_ML_RUNTIME

docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.10-cuda:2024.05.1-b8


#### 2.3 현재 작업 중인 프로젝트 정보 가져오기
현재 작업 중인 프로젝트의 메타데이터 가져오기 및 인쇄

In [4]:
# Get the identifier of the current project
project = client.get_project(project_id=os.getenv("CDSW_PROJECT_ID"))

In [5]:
print(f'Current working project name: {project.name}')

Current working project name: Hands on Lab Workshop with LLM - hyeongyu_yeo1


### 2.4 Chroma Vector DB를 채우기 위한 작업 생성 및 실행
이 코드는 등록한 job을 구분하기 위해 임의의 식별자를 생성하고, 프로젝트 ID, 스크립트, 리소스 지정 등 job 등록에 필요한 여러 파라미터를 설정합니다.
설정한 파라미터로  Chroma Vector DB를 채우기 위한 job을 생성하고, 생성된 job을 실행합니다.

In [6]:
random_id=''.join(random.choice(string.ascii_lowercase) for i in range(10))
job_body = cmlapi.CreateJobRequest(
    project_id = project.id,
    name = "Populate Chroma Vector DB " + random_id, 
    script = "2_populate_vector_db/populate_chroma_vectors.py",
    cpu = 1,
    memory = 4,
    runtime_identifier = os.getenv('JOB_IMAGE_ML_RUNTIME')
)

job_result = client.create_job(
    body = job_body, 
    project_id = str(project.id)
)

job_run = client.create_job_run(
    cmlapi.CreateJobRunRequest(),
    project_id = project.id, 
    job_id = job_result.id
)

In [12]:
job_run.scheduling_at

datetime.datetime(2024, 12, 17, 17, 49, 7, 856728, tzinfo=tzlocal())


**(보너스)** 실행 후 Job 매뉴로 들어가서 새로 등록한 job이 실행되는지 확인해 보세요. 아래 처럼 Running 상태로 보이면 정상 실행되는 상태입니다. <br>
**(보너스)** chroma_vectordb_usage.ipynb 파일을 chroma vector db에 추가된 문서 내용을 확인해 보세요

![CML API를 사용한 JOB 등록](../assets/job-run-cmlapi.png)